##**Branch and Bound General Method**


### **Branch and Bound General Method**

The **Branch and Bound (B&B)** technique is a systematic way to solve optimization problems, particularly those involving combinatorial structures like the 0/1 knapsack problem, traveling salesperson problem, and more. It is particularly useful for solving problems where a brute-force approach would be computationally expensive.

---

### **Key Concepts of Branch and Bound**

1. **Search Space**: The solution space is represented as a tree where each node represents a partial solution.
2. **Bounding**: Calculate a bound (upper or lower) for the best possible solution in a branch. If the bound is worse than the current best solution, prune the branch.
3. **Branching**: Divide the problem into smaller subproblems by branching on choices (e.g., include or exclude an item in the knapsack).
4. **Queue or Stack**: Maintain a list of nodes to explore, typically implemented as a priority queue, stack, or queue, depending on the problem.

---

### **General Steps of Branch and Bound**

1. **Initialize**:
   - Define the root node (e.g., empty solution).
   - Set initial bounds (e.g., worst-case values).

2. **Expand Nodes**:
   - Start with the root node and expand it by branching on decisions (e.g., include/exclude items).
   - Generate child nodes for the next level.

3. **Bounding**:
   - For each node, calculate an upper bound (maximum possible solution) or a lower bound (minimum guaranteed solution).
   - If the bound of a node is worse than the current best solution, prune the branch.

4. **Update Best Solution**:
   - If a complete solution is found (leaf node), update the current best solution.

5. **Select Next Node**:
   - Use a priority queue to explore nodes with better bounds first (best-first search).
   - Alternatively, use depth-first or breadth-first strategies for node selection.

6. **Terminate**:
   - Stop when all nodes have been explored or pruned.

---

### **Branch and Bound Pseudocode**

```plaintext
function BranchAndBound(root):
    Initialize best_solution = None
    Initialize best_value = -∞
    Add root to a priority queue PQ

    while PQ is not empty:
        node = PQ.pop()

        if node is a complete solution:
            if node.value > best_value:
                best_solution = node
                best_value = node.value
        else:
            for child in expand(node):
                if child.bound > best_value:
                    PQ.add(child)

    return best_solution
```

---

### **Branch and Bound for 0/1 Knapsack**

#### Problem:
- **Input**: Items with weights and values, knapsack capacity.
- **Output**: Maximum value that can be obtained without exceeding the capacity.

#### Bounding:
- Calculate the maximum value achievable from a node by assuming fractional items (similar to the greedy approach).

---
### **Example**

#### Input:
- **Weights**: \([1, 2, 3, 2]\)
- **Values**: \([10, 15, 40, 25]\)
- **Capacity**: \(5\)

#### Output:
Maximum value: 65

Items included (0-based indices): [2, 3]

---

### **Time Complexity**

- **Worst-case complexity**: \($O(2^n$)\), where \($n$\) is the number of items.
  - Pruning reduces the number of branches explored compared to backtracking, but the problem is still exponential in nature.
- **Best-case complexity**: Depends on the effectiveness of pruning.

---

### **Space Complexity**

- **Auxiliary space**: \($O(2^n)$\) for the priority queue in the worst case.

---

## **Branch and Bound for 0/1 Knapsack**

In [ ]:

### **Python Implementation for 0/1 Knapsack**

import heapq

class Node:
    def __init__(self, level, value, weight, bound, items):
        self.level = level  # Depth in the tree (which item is being considered)
        self.value = value  # Current total value
        self.weight = weight  # Current total weight
        self.bound = bound  # Upper bound of maximum value achievable
        self.items = items  # Items included so far

    def __lt__(self, other):
        return self.bound > other.bound  # Priority queue prefers higher bounds

def knapsack_branch_and_bound(weights, values, capacity):
    def calculate_bound(node):
        if node.weight >= capacity:
            return 0  # Infeasible, no bound
        bound = node.value
        total_weight = node.weight
        level = node.level + 1

        # Add fractional items to bound
        while level < len(weights) and total_weight + weights[level] <= capacity:
            total_weight += weights[level]
            bound += values[level]
            level += 1

        if level < len(weights):
            bound += (capacity - total_weight) * (values[level] / weights[level])

        return bound

    # Initialize priority queue
    pq = []
    root = Node(level=-1, value=0, weight=0, bound=0, items=[])
    root.bound = calculate_bound(root)
    heapq.heappush(pq, root)

    max_value = 0
    best_items = []

    while pq:
        current = heapq.heappop(pq)

        # If the bound of the current node is not better than max_value, skip
        if current.bound <= max_value:
            continue

        # Generate child nodes
        level = current.level + 1
        if level < len(weights):
            # Case 1: Include the current item
            include = Node(
                level=level,
                value=current.value + values[level],
                weight=current.weight + weights[level],
                bound=0,
                items=current.items + [level]
            )
            if include.weight <= capacity and include.value > max_value:
                max_value = include.value
                best_items = include.items
            include.bound = calculate_bound(include)
            if include.bound > max_value:
                heapq.heappush(pq, include)

            # Case 2: Exclude the current item
            exclude = Node(
                level=level,
                value=current.value,
                weight=current.weight,
                bound=0,
                items=current.items
            )
            exclude.bound = calculate_bound(exclude)
            if exclude.bound > max_value:
                heapq.heappush(pq, exclude)

    return max_value, best_items

# Example Usage
if __name__ == "__main__":
    weights = [1, 2, 3, 2]
    values = [10, 15, 40, 25]
    capacity = 5

    max_val, selected_items = knapsack_branch_and_bound(weights, values, capacity)
    print(f"Maximum value: {max_val}")
    print(f"Items included (0-based indices): {selected_items}")

Maximum value: 65
Items included (0-based indices): [2, 3]


### **Traveling Salesperson Problem (TSP) Using Branch and Bound**

The **Traveling Salesperson Problem (TSP)** aims to find the shortest possible route that visits each city exactly once and returns to the starting city. The **Branch and Bound (B&B)** method provides an optimal solution by systematically exploring possible tours and pruning suboptimal paths based on cost bounds.

---

### **Key Concepts**

1. **Search Space**: Represented as a tree where:
   - Each level represents a city.
   - Each node represents a partial tour.
2. **Bounding**: Use a lower bound (minimum cost estimate) to prune branches that cannot lead to a better solution.
3. **Priority Queue**: Explore branches in increasing order of cost to find the optimal solution faster.

---

### **Steps in B&B for TSP**

1. **Define the Root Node**:
   - Start at any city.
   - Calculate a bound on the minimum possible cost from this node.

2. **Branching**:
   - Generate child nodes by visiting unvisited cities.
   - Each child represents a partial tour.

3. **Bounding**:
   - Compute a lower bound for each child node using a heuristic (e.g., reduced cost matrix).
   - Prune branches whose bound is greater than the current best solution.

4. **Priority Queue**:
   - Maintain a priority queue of nodes to explore, sorted by their lower bounds.

5. **Termination**:
   - Stop when all branches have been explored or pruned.
   - The current best solution is the optimal tour.

---

### **Lower Bound Calculation**
- Use a **reduced cost matrix**:
  - Subtract the smallest value in each row and column from the matrix.
  - The sum of these reductions is a lower bound for the cost.
---

### **Example**

#### Input:
Cost Matrix (4 cities):

∞   10   15   20

10   ∞   35   25

15   35   ∞   30

20   25   30   ∞

#### Output:
Minimum cost: 80
Optimal path: [0, 1, 3, 2, 0]

---

### **Explanation of the Output**
1. **Optimal Path**: The tour starts at city \($0$\), visits cities \($1 \to 3 \to 2$\), and returns to \($0$\).
2. **Minimum Cost**: The total cost for this tour is \(80\).

---

### **Time Complexity**

- **Worst-case complexity**: \($O(n!$)\), where \($n$\) is the number of cities.
  - Due to the factorial number of permutations in the worst-case scenario.
- **Best-case complexity**: Dependent on the effectiveness of bounding and pruning.

### **Space Complexity**

- **Auxiliary space**: \($O(n!$)\) for the priority queue.

---

### **Advantages**
1. Provides the optimal solution.
2. Efficient pruning reduces the number of explored branches compared to brute-force.

---

### **Visualization**
For a better understanding, a tree-like structure can be drawn:
- **Root**: City 0.
- **Branches**: Partial tours from each node.
- **Leaves**: Complete tours.

In [ ]:
import heapq

class Node:
    def __init__(self, level, path, cost, bound):
        self.level = level
        self.path = path
        self.cost = cost
        self.bound = bound

    def __lt__(self, other):
        return self.bound < other.bound

def calculate_bound(cost_matrix, path):
    n = len(cost_matrix)
    bound = 0

    # Calculate cost of the current path
    for i in range(len(path) - 1):
        bound += cost_matrix[path[i]][path[i + 1]]

    # Estimate cost for remaining unvisited cities
    unvisited = [i for i in range(n) if i not in path]
    for city in unvisited:
        min_cost = float('inf')
        for next_city in range(n):
            if next_city != city and next_city not in path:
                min_cost = min(min_cost, cost_matrix[city][next_city])
        if min_cost != float('inf'):  # Add only if a valid edge is found
            bound += min_cost

    return bound

def tsp_branch_and_bound(cost_matrix):
    n = len(cost_matrix)
    pq = []
    root = Node(level=0, path=[0], cost=0, bound=calculate_bound(cost_matrix, [0]))
    heapq.heappush(pq, root)

    min_cost = float('inf')
    best_path = None

    while pq:
        current = heapq.heappop(pq)

        # If the bound is greater than the current minimum cost, prune this branch
        if current.bound >= min_cost:
            continue

        # If the node represents a complete tour, update the minimum cost
        if current.level == n - 1:
            complete_cost = current.cost + cost_matrix[current.path[-1]][current.path[0]]
            if complete_cost < min_cost:
                min_cost = complete_cost
                best_path = current.path + [0]
            continue

        # Branch to explore the next cities
        for i in range(n):
            if i not in current.path:
                new_path = current.path + [i]
                new_cost = current.cost + cost_matrix[current.path[-1]][i]
                new_bound = calculate_bound(cost_matrix, new_path)
                if new_bound < min_cost:
                    new_node = Node(level=current.level + 1, path=new_path, cost=new_cost, bound=new_bound)
                    heapq.heappush(pq, new_node)

    return min_cost, best_path

# Example Usage
if __name__ == "__main__":
    cost_matrix = [
        [float('inf'), 10, 15, 20],
        [10, float('inf'), 35, 25],
        [15, 35, float('inf'), 30],
        [20, 25, 30, float('inf')],
    ]

    min_cost, best_path = tsp_branch_and_bound(cost_matrix)
    print(f"Minimum cost: {min_cost}")
    print(f"Optimal path: {best_path}")

Minimum cost: 80
Optimal path: [0, 1, 3, 2, 0]
